# VI. Interactive Visualization


# VI. Веб-карта с Folium


Folium — это библиотека Python для создания интерактивных веб-карт на основе Leaflet.js. С её помощью можно легко визуализировать геопространственные данные, добавлять маркеры, всплывающие подсказки, хлороплетные карты, а также расширять функциональность с помощью плагинов.

**Основные возможности Folium**:

- Создание интерактивных карт с набором нескольких слоев
- Добавление маркеров, кружков, всплывающих подсказок и кластеров
- Построение картограмм (Choropleth)
- Подключение плагинов (MiniMap, MarkerCluster, HeatMap и др.)

В это разделе мы создадим интерактивную карту театров Санкт-Петербурга


Импортируем библиотеки


In [ ]:
import folium
import geopandas as gpd

## 0. Подготовка данных


Читаем данные


In [ ]:
theaters = gpd.read_file('data/spb_theaters.geojson')
admin_district = gpd.read_file('data/spb_admin.gpkg', layer="okrug")


Посчитаем кол-во театров по районам города


In [ ]:
# приведем данные в одну систему координат
if theaters.crs != admin_district.crs:
    theaters = theaters.to_crs(admin_district.crs)

# Пространственное пересечение
theaters_within_district = gpd.sjoin(theaters, admin_district, how="left", predicate="within")

# Группировка и подсчёт
theaters_count = theaters_within_district.groupby('NAME').size().reset_index(name='theaters_count')

# Соединяем данные со слоем с районами по полю  NAME
admin_district_with_count = admin_district.merge(
    theaters_count,
    on='NAME', 
    how='left'
)

# Посчитаем кол-во театров на 100 000 ччеловек
admin_district_with_count['theatersPerPop'] = admin_district_with_count['theaters_count']/(admin_district_with_count['Popul']/100000)


# Поcмотрим на данные
admin_district_with_count.head()


## 1. Настройка карты

- Внимательно рассмотрите функцию, попробуйте понять, за что отвечает каждая переменная


In [ ]:
data = admin_district_with_count.to_crs('EPSG:4326')
m = folium.Map(location=[data.centroid.y.mean(), data.centroid.x.mean()], zoom_start=10,  tiles="cartodb positron", control_scale=True)


- Изучите область карты. Понадобятся ли изменения?


In [ ]:
m

## 2. Добавление картограммы


In [ ]:
folium.Choropleth(
    name='Theaters Choropleth',
    geo_data=admin_district_with_count,
    data=admin_district_with_count,
    columns=['NAME', 'theatersPerPop'],
    fill_color='YlGnBu',
    fill_opacity = 0.5,
    key_on='feature.properties.NAME',
    nan_fill_color='lightgray',
    nan_fill_opacity=0.4,
   line_color = "white",
   legend_name="Theaters per 100 000 "
).add_to(m)

- Посмотрим на карту


In [ ]:
m

## 3. Добавление подсказки (tooltip)


In [ ]:
folium.GeoJson(
    admin_district_with_count,
    name="Districts with tooltips",
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'transparent',
        'weight': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME', 'theaters_count'],
        aliases=['Округ:', 'Количество театров:'],
        localize=True
    )
).add_to(m)

In [ ]:
m

- Посмотрим на карту


## 4. Добавление точек в виде маркеров и их кластеризация


- Импортируем плагин


In [ ]:
from folium.plugins import MarkerCluster, FeatureGroupSubGroup



- Добавляем маркеры и возможность их кластеризации


In [ ]:
theaters = theaters[~(theaters.geometry.isna() | theaters.is_empty)]

# Создаём MarkerCluster и SubGroup
marker_cluster = MarkerCluster(name='Theaters').add_to(m)
mc1 = FeatureGroupSubGroup(marker_cluster, 'Theaters').add_to(m)

# Добавляем каждый театр как Marker
for idx, row in theaters.iterrows():
    # Координаты
    lat = row.geometry.y
    lon = row.geometry.x
    
    # Пример popup: имя театра + адрес
    popup_text = row['name']
    
    # Tooltip при наведении (можно упростить)
    tooltip_text = row['name']
    
    # Маркер с кастомной иконкой
    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        tooltip=tooltip_text,
        icon=folium.Icon(icon='heart', prefix='glyphicon', color='darkblue')
    ).add_to(mc1)

- Посмотрим на карту


In [ ]:
m

## 5. Добавление плагинов/виджетов


- Импортируем плагины


In [ ]:
from folium.plugins import MousePosition
from folium.plugins import Fullscreen

- Добавляем контроль слоёв


In [ ]:
folium.LayerControl().add_to(m)

- Добавлеям координаты курсора


In [ ]:
MousePosition().add_to(m)


- Добавлеям полноэкранный режим


In [ ]:
Fullscreen(
    position="bottomright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

- Смотрим на карту


In [ ]:
m

## 6. Сохраняем результат


In [ ]:
m.save("index.html")

<p style="color:#58568E; font-style:italic">The module is in progress</p>
